# Miscellaneous notes from during development

by Jessica Scheick

In [ ]:
ssh jscheick@pennell.ess.uci.edu
cd /u/pennell-z0/eric/GREENLAND/ArticDEM/KANE_BASIN

## Xarray

In [ ]:
ds['elevation'].groupby('dtime').max(['x','y'])

### Convert from ellipsoid heights to geoid heights in Xarray
**Note: this is now done as a pre-processing step using gdalwarp. The pointwise computation implemented below takes unreasonably long.**

Currently the EPSG codes for the transformation are hard-coded in.

In [ ]:
# ds=ds.bergxr.to_geoid()

## Holoviews

In [17]:
hv.help(hv.Image)

Image

Online example: http://holoviews.org/reference/elements/bokeh/Image.html

-------------
Style Options
-------------

	alpha, cmap, muted, visible

(Consult bokeh's documentation for more information.)

------------
Plot Options
------------

The plot options are the parameters of the plotting class:

Parameters of 'RasterPlot'

Parameters changed from their default values are marked in red.
Soft bound values are marked in cyan.
C/V= Constant/Variable, RO/RW = ReadOnly/ReadWrite, AN=Allow None

Name                                 Value                         Type         Bounds     Mode  

active_tools                           []                          List       (0, None)    V RW  
align                                 None                    ObjectSelector               V RW  
apply_extents                         True                       Boolean        (0, 1)     V RW  
apply_ranges                          True                       Boolean        (0, 1)     V RW  
asp

### Using Holoviews to create land-masked plots

As described in the next section and in my notes, Holoviews is not yet up to the geospatial plotting tasks I'm asking of it. I switched to running the 2m resolution data (from 50m) and, among other issues, I can no longer create a land mask. My original code to do that is now below, and I'm going to shift to static plots within the workflow notebook.

In [7]:
# NOTE: can't run this cell for the 2m DEMs - it's too processing intensive

ds['alpha'] = np.abs(ds['land_mask']-1)

# Cannot get the colors to work with Bokeh: https://stackoverflow.com/questions/64427909/holoviews-heatmap-specify-color-for-each-point/64463899#64427909
# import bokeh.models
# bokehcmap = bokeh.models.LinearColorMapper(palette=['red', 'blue'], low=0, high=1)

# The below two lines work for the colors, but not the alpha (see warning).
# Also, they use matplotlib as a backend, but maybe only sometimes?
# mycmap = mpl.colors.ListedColormap([(0.5, 0.35, 0.35, 1.), (0.5, 0., 0.6, 0)])#['brown'/#7f645a,'purple'/#7f0559])
# land = ds.hvplot(x='x', y='y', z='land_mask', aspect='equal')
# land.opts(cmap=mycmap, alpha=hv.dim('alpha'))#, colorbar=False)
# land

# I'm not the biggest fan of the shade of brown or the splotchiness, but it's closer to what I want at least
land = hv.HeatMap(ds, kdims=['x','y'], vdims=['land_mask','alpha'])
land.opts(aspect='equal',alpha=hv.dim('alpha'), cmap=['#7f645a','purple'], show_grid=False)

In [ ]:
# ds['elevation_orig'] = ds['elevation']
ds['elevation'] = ds['elevation'].where(ds.land_mask == True)

In [ ]:
scrolldem = ds['elevation'].hvplot.image(x='x', y='y',rasterize=True, aspect='equal', cmap='magma', #dynamic=False
                       xlabel="x (km)", ylabel="y (km)")
scrolldem*land

### Using Holoviews to create a contour plot of iceberg outlines for a single DEM

In [ ]:
timei=1
print(ds['dtime'].isel({'dtime':timei}))

In [ ]:
dsdem = ds['elevation'].isel({'dtime':timei}).hvplot.image(x='x', y='y',rasterize=True, aspect='equal', cmap='magma', #dynamic=False
                       xlabel="x (km)", ylabel="y (km)")
# dsdem

In [ ]:
berglines = hv.Contours(ds['berg_outlines'].values[timei])
berglines.opts(aspect="equal", color='gray')
print(berglines)
# berglines

In [ ]:
bergsr = (dsdem+berglines)
# bergsr.opts(width=1000)
# bergsr

In [ ]:
dsdem*berglines*land

### Manipulating and Updating Options (e.g. to format multi-panel plots)

NOTE: clearing bergsr (`bergsr.opts.clear()`) under the hood clears dem.opts and berglines.opts but DOES NOT clear bergsr.opts
I've filed a bug report on this (Oct 2020)

In [ ]:
dem.opts.info()
berglines.opts.info()
bergsr.opts.info()
bergszdist.opts.info()

In [ ]:
# figure = pn.Column(pn.Row(dem+berglines, width=500), pn.Row(bergszdist, width=500))
# works for lining things up if you haven't also set frame_width anywhere. If you have, you'll need to remove that opt.
# I'm not sure how to remove just one opt, so might need to do hvobj.opts.clear()
# a key lesson here is the difference between opts and options. opts sets the values FOR THE OBJECT, and they're carried through whenever that object is used
# options sets the values temporarily for the current run, so the values won't be carried through
# Thus, if we do bergszdist.opts.info() after running the below, it will not have width=1000 because it was only set as an option
# This is useful for making things like axis labels and aspect ratios "mandatory" while allowing flexibility in plotting
# The suggested way to do this in the docs is to use clone=True (by default clone=False) to .opts...

pnfig = pn.Column(
    pn.Row(
        bergsr.opts(width=1000, clone=True)), # note: need to pre-combine these two figures to have a joint width or run into width vs frame_width issues
    bergszdist.opts(width=1000, shared_axes=False, clone=True))
pnfig

### Attempts at Plotting Iceberg Outlines using Holoviews (e.g. to have a slider bar)

After lots of struggle, I had to give up trying to have a slider bar that allowed you to see the DEM (elevations) and iceberg outlines side by side. Instead, it'll have to be one date at a time.

I tried:
- using groupby along dtime
- creating a separate dataset of iceberg outlines to get the "values" from the DataArray
- using the built in hv.Contours (with a contour value at threshold)
- using hv.Contours and hv.Polygons
- converting to a holoviews dataset and using `.to` to generate the contours

In [7]:
#find elevation contours at example threshold and filter (for a single time value)
threshold=15
bergsdir = raster_ops.poly_from_thresh(ds.x.values, ds.y.values, ds['elevation'].isel({'dtime':0}), threshold=threshold)

In [9]:
print(type(bergsdir))
# print(bergsdir)
berglinesdir = hv.Contours(bergsdir) #ds['berg_outlines'].groupby('dtime'))
berglinesdir.opts(aspect="equal", color='k')
print(berglinesdir)
berglinesdir

<class 'list'>
:Contours   [x,y]


:Contours   [x,y]

In [34]:
# for a single timestamp but using the already-calculated iceberg outlines in the dataArray
bergs = [ds['berg_outlines'].isel({'dtime': [i]}).values for i in range(0,len(ds.dtime.values))]
# bergs = ds['berg_outlines'].values[0][0]#.groupby('dtime')

print(type(bergs))
print(len(bergs))
# print(type(bergs[0]))
# print(len(bergs[0]))
# print(bergs[0])
berglines = hv.Contours(ds['berg_outlines'].values[0])#.groupby('dtime'))]
# berglines = hv.Contours(bergs)
berglines.opts(aspect="equal", color='k')
print(berglines)
berglines

<class 'list'>
2
:Contours   [x,y]


:Contours   [x,y]

In [30]:
demcont = hv.operation.contours(dsdem, levels=[10]) #if levels is given as an int, it's how many levels it gets divided into. If it's given as a list, it contours at those scalar values.
# demcont.opts(aspect='equal')
demshow = dsdem + demcont
# demshow.opts()
demshow

:Layout
   .DynamicMap.I  :DynamicMap   [dtime]
      :Image   [x,y]   (elevation)
   .DynamicMap.II :DynamicMap   [dtime]
      :Contours   [x,y]   (elevation)